In [20]:
#Goal -> Evaluate the potential homebuyer data and determine if they are ready to buy a home

In [21]:
#Imports
import pandas as pd

In [22]:
#Load the data
all_data = pd.read_csv('HackUTD-2023-HomeBuyerInfo.csv')
all_data.head()

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615


In [23]:
#Calculate LTV
all_data['LTV'] = all_data['LoanAmount'] / all_data['AppraisedValue']


#Add PMI to MonthlyMortgagePayment based on LTV
def add_pmi(row):
    if row['LTV'] >= .8:
        return row['MonthlyMortgagePayment'] + row['AppraisedValue'] * .01 / 12
    else:
        return row['MonthlyMortgagePayment']
    
all_data['MonthlyMortgagePaymentProcessed'] = all_data.apply(add_pmi, axis = 1)
all_data.head()

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyMortgagePaymentProcessed
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778,0.88,1351.623333
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546,0.91,2519.555833
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736,0.92,2661.954167
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761,0.93,1723.860000
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615,0.95,3381.274167


In [24]:
#Calculate DTI
all_data['DTI'] = (all_data['CreditCardPayment'] + all_data['CarPayment'] + all_data['StudentLoanPayments'] + all_data['MonthlyMortgagePaymentProcessed']) / all_data['GrossMonthlyIncome']

#Calculate FEDTI
all_data['FEDTI'] = all_data['MonthlyMortgagePaymentProcessed'] / all_data['GrossMonthlyIncome']

In [27]:
#Determine approval
def add_filter(df):
    approved_list = []
    lti_list = []
    credit_score_list = []
    dti_43_list = []
    dti_36_list = []
    fedti_list = []

    for index, row in df.iterrows():
        approved = 'Y'
        note = []
        credit_score, lti, dti_43, dti_36, fedti = 0, 0, 0, 0, 0

        if row['CreditScore'] < 640:
            approved = 'N'
            credit_score = 1
        if row['LTV'] >= .8:
            lti = 1
        if row['DTI'] >= .43:
            approved = 'N'
            dti_43 = 1
        elif row['DTI'] >= .36:
            approved = 'N'
            dti_36 = 1
        if row['FEDTI'] >= .28:
            approved ='N'
            fedti = 1

        approved_list.append(approved)
        credit_score_list.append(credit_score)
        lti_list.append(lti)
        dti_43_list.append(dti_43)
        dti_36_list.append(dti_36)
        fedti_list.append(fedti)

    df['Approved'] = approved_list
    df['credit_score_check'] = credit_score_list
    df['lti_check'] = lti_list
    df['dti_43_check'] = dti_43_list
    df['dti_36_check'] = dti_36_list
    df['fedti_check'] = fedti_list
    
    return df

In [28]:
all_data = add_filter(all_data)
all_data.head()

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyMortgagePaymentProcessed,DTI,FEDTI,Approved,credit_score_check,lti_check,dti_43_check,dti_36_check,fedti_check
0,1,3103.0,317.0,374.0,250.0,268468.0,32216.16,236251.84,1127.90,778,0.88,1351.623333,0.738841,0.435586,N,0,1,1,0,1
1,2,2939.0,368.0,395.0,228.0,335467.0,30192.03,305274.97,2240.00,546,0.91,2519.555833,1.194473,0.857283,N,1,1,1,0,1
2,3,4676.0,222.0,352.0,368.0,461177.0,36894.16,424282.84,2277.64,736,0.92,2661.954167,0.770734,0.569280,N,0,1,1,0,1
3,4,8469.0,224.0,416.0,400.0,326904.0,22883.28,304020.72,1451.44,761,0.93,1723.860000,0.326350,0.203549,Y,0,1,0,0,0
4,5,2406.0,497.0,385.0,444.0,472661.0,23633.05,449027.95,2987.39,615,0.95,3381.274167,1.956473,1.405351,N,1,1,1,0,1
